In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from config import rmax , aplha , beta


class Sensor:
    def __init__(self, robot_position, map_matrix):
        """Initialize sensors.
        
        Params
        ======
            robot_position (tuple): [y_robot, x_robot, theta_robot]
            map_matrix (numpy array): matrix representing the map
        """
        self.fov = np.pi
        self.angle_step = 0.008
        self.measure_phi = np.linspace(-self.fov, self.fov, self.angle_step)
        self.rmax = rmax
        self.robot_position = robot_position
        self.map_matrix = map_matrix
        self.map_height, self.map_widht = map_matrix.shape
    
    def get_LIDAR_measurement(self):
        """Simulate LIDAR measurement.
        
        Returns
        =======
            (numpy array): array of range measurements
        """
        height, width = self.map_matrix.shape
        meas_phi = np.linspace(-self.fov/2, self.fov/2, 9)
        meas_r = rmax * np.ones(meas_phi.shape)

        # Vectorized computation of cell coordinates
        r_values = np.arange(2, rmax)
        xi = self.robot_position[1] + np.outer(r_values, np.cos(self.robot_position[2] + meas_phi)).astype(int)
        yi = self.robot_position[0] + np.outer(r_values, np.sin(self.robot_position[2] + meas_phi)).astype(int)

        # Mask for valid coordinates
        valid_mask = (xi >= 0) & (xi < height) & (yi >= 0) & (yi < width)

        # Map look-up
        for i in range(len(meas_phi)):
            valid_cells = valid_mask[:, i]
            xi_valid = xi[valid_cells, i]
            yi_valid = yi[valid_cells, i]

            # Find the first occupied cell or out-of-bound
            occupied_cells = (self.map_matrix[yi_valid, xi_valid] == 1)
            if occupied_cells.any():
                meas_r[i] = r_values[valid_cells][occupied_cells.argmax()]
            else:
                meas_r[i] = r_values[valid_cells][-1] if valid_cells.any() else rmax
        return meas_r
